# Oxygen interstitial Defect(s) in SnO.

This example demonstrates how to create a O interstitial defect in a SnO material. Following the publication:

> **A. Togo, F. Oba, and I. Tanaka**
> "First-principles calculations of native defects in tin monoxide"
> Phys. Rev. B 74, 195128 (2006)
> [DOI: 10.1103/PhysRevB.74.195128](https://doi.org/10.1103/PhysRevB.74.195128)

Focusing on the Fig 4. a):

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/defects/defect_point_interstitial_tin_oxide/0-figure-from-manuscript.webp" alt="SnO with O-interstitial" width="400"/>

!Note: Voronoi site placement method is used for the interstitial defect, for a precise reproduction relaxation is needed.

## 1. Prepare the Environment
### 1.1. Set up defects parameters 
Defect Configuration parameters are described in [Defect Configuration](https://github.com/Exabyte-io/made/blob/8196b759242551c77d1791bf5bd2f4150763cfef/src/py/mat3ra/made/tools/build/defect/configuration.py#L102).

In [ ]:
MATERIAL_NAME = "SnO"
SUPERCELL_MATRIX = [[2, 0, 0], [0, 2, 0], [0, 0, 2]]

# List of dictionaries with defect parameters
DEFECT_CONFIGS = [
    {
        "type": "vacancy",
        "coordinate": [0.0, 0.25, 0.525],
        "placement_method": "closest_site",
    },
    {
        "type": "interstitial",
        "coordinate": [0.0, 0.25, 0.35],
        "element": "O",
        "placement_method": "voronoi_site"
    }
]

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples|create_point_defect.ipynb")

### 1.3. Get input material

In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials

material = Material.create(Materials.get_by_name_first_match(MATERIAL_NAME))

### 1.4. Create and preview Supercell

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.helpers import create_supercell

supercell = create_supercell(material, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create the Defects

### 2.1. Prepare defect dictionaries and validate them

In [ ]:
from mat3ra.made.tools.build.defective_structures.zero_dimensional.point_defect.types import PointDefectDict

defect_dicts = [PointDefectDict(**{
    "type": defect_config["type"],
    "coordinate": defect_config["coordinate"],
    "element": defect_config.get("element", None),
    "placement_method": defect_config["placement_method"]
}) for defect_config in DEFECT_CONFIGS]

### 2.2. Create the defects

In [ ]:
from mat3ra.made.tools.helpers import create_multiple_defects

material_with_defect = create_multiple_defects(
    supercell,
    defect_dicts=defect_dicts,
)

## 3. Visualize Result(s)

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}],
          viewer="wave")

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials, download_content_to_file

material_with_defect.name = "Sn15 O17, Interstitial O Defect"
set_materials([material_with_defect])
download_content_to_file(material_with_defect.to_json(), f"{material_with_defect.name}.json")